In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from wordcloud import WordCloud

In [2]:
np.random.seed(42)
random.seed(42)

In [3]:
addresses = pd.read_csv('DATA/tbl_addresses.csv')
customer_acc = pd.read_csv('DATA/tbl_customer_accounts.csv')
customer_rew = pd.read_csv('DATA/tbl_customer_reviews.csv')
customers = pd.read_csv('DATA/tbl_customers.csv')
reviews = pd.read_csv('DATA/tbl_labelled_reviews.csv')
orders = pd.read_csv('DATA/tbl_orders.csv')
products = pd.read_csv('DATA/tbl_products.csv')

### tbl_customers

In [4]:
customers['job_type'] = customers['job_type'].fillna('Unknown')
customers['email_provider'] = customers['email_provider'].fillna('Unknown')
customers['flag_phone_provided'] = customers['flag_phone_provided'].fillna(0).astype(bool)
customers

,customer_id,address_id,birthdate,gender,job_type,email_provider,flag_phone_provided,flag_privacy
0,891057,863914,2001-10-22,F,Unknown,yahoo.it,True,True
1,655484,639856,2001-04-30,M,Unknown,yahoo.it,True,False
2,514958,494806,2001-01-01,M,Unknown,libero.it,True,True
3,398355,377261,2000-03-09,M,Unknown,gmail.com,True,True
4,23808,20227,1999-11-21,F,Unknown,virgilio.it,True,True
...,...,...,...,...,...,...,...,...
140941,449222,428121,1945-05-30,F,Unknown,libero.it,True,True
140942,368113,346970,1943-11-18,F,Unknown,yahoo.com,True,True
140943,424451,403292,1943-05-07,F,Unknown,gmail.com,True,True
140944,808137,789737,1943-02-13,F,Unknown,yahoo.it,True,True


### customer_acc

In [5]:
customer_acc['activation_date'] = pd.to_datetime(customer_acc['activation_date'])
customer_acc

,customer_id,account_id,favorite_store,loyalty_type,loyatlty_status,activation_date
0,9557,746573,9,premium,1,2023-01-11
1,743090,776971,2,premium,1,2023-01-30
2,813156,791681,45,premium,1,2023-02-08
3,843392,825439,1,premium,1,2023-03-01
4,20476,815010,16,premium,1,2023-02-23
...,...,...,...,...,...,...
169489,828458,895971,47,standard,1,2023-04-09
169490,575891,525794,1,standard,1,2022-09-14
169491,316991,231471,14,standard,1,2022-04-15
169492,403638,329448,1,standard,1,2022-05-31


### addresses

In [6]:
missing_values = addresses.isnull().sum()
missing_values

address_id         0
postal_code       49
district       19066
region          6746
dtype: int64

In [7]:
def fill_missing_location_data(addresses):
    postal_code_map = addresses.dropna(subset=['postal_code', 'district', 'region']) \
        .groupby('postal_code').agg({
        'district': lambda x: x.value_counts().index[0],
        'region': lambda x: x.value_counts().index[0]
    }).to_dict('index')

    def fill_from_map(row):
        if pd.isnull(row['district']) or pd.isnull(row['region']):
            mapped_values = postal_code_map.get(row['postal_code'], {})
            row['district'] = mapped_values.get('district', row['district'])
            row['region'] = mapped_values.get('region', row['region'])
        return row

    df = addresses.apply(fill_from_map, axis=1)

    imputer = SimpleImputer(strategy='most_frequent')
    df[['district', 'region']] = imputer.fit_transform(df[['district', 'region']])

    return df

df = fill_missing_location_data(addresses)

print(df[['postal_code', 'district', 'region']].isnull().sum())
addresses = df

postal_code    49
district        0
region          0
dtype: int64


In [8]:
unique_combinations = addresses.groupby(['postal_code', 'district', 'region']).size().reset_index(name='count')
print("Unique combination:", len(unique_combinations))

# Проверяем несоответствия
inconsistencies = unique_combinations[unique_combinations['count'] < 10]
print("\nrare combination:")
print(inconsistencies)

Unique combination: 4668

rare combination:
      postal_code district          region  count
0             0.0       BG       LOMBARDIA      1
1             0.0       BO  EMILIA ROMAGNA      1
3             0.0       TV          VENETO      1
4             0.0       VA       LOMBARDIA      2
8            12.0       RO          VENETO      1
...           ...      ...             ...    ...
4663      98167.0       ME         SICILIA      2
4664      98168.0       ME         SICILIA      8
4665      99000.0       NZ       LOMBARDIA      1
4666      99999.0       EE       LOMBARDIA      1
4667      99999.0       MI       LOMBARDIA      1

[2555 rows x 4 columns]


In [10]:
def fill_missing_postal_codes(df):
    # 1. Заполнение на основе district и region
    district_region_to_postal = df.dropna().groupby(['district', 'region'])['postal_code'].agg(
        lambda x: x.value_counts().index[0]
    ).to_dict()

    df['postal_code'] = df.apply(
        lambda row: district_region_to_postal.get((row['district'], row['region']), row['postal_code'])
        if pd.isnull(row['postal_code']) else row['postal_code'],
        axis=1
    )

    # 2. KNN Imputer для оставшихся пропусков
    if df['postal_code'].isnull().sum() > 0:
        # Преобразуем postal_code в числовой формат
        df['postal_code_num'] = pd.to_numeric(df['postal_code'], errors='coerce')

        # Выбираем признаки для импутации
        features_for_imputation = ['postal_code_num', 'latitude', 'longitude']  # предполагаем, что у нас есть координаты

        # Создаем и применяем KNN Imputer
        imputer = KNNImputer(n_neighbors=5)
        df[features_for_imputation] = imputer.fit_transform(df[features_for_imputation])

        # Преобразуем обратно в строковый формат
        df['postal_code'] = df['postal_code_num'].apply(lambda x: f"{x:05.0f}" if not pd.isnull(x) else None)
        df.drop('postal_code_num', axis=1, inplace=True)

    # 3. Для оставшихся пропусков (если есть) используем специальное значение
    df['postal_code'].fillna('UNKNOWN', inplace=True)

    return df

# Применяем функцию к нашему DataFrame
df = fill_missing_postal_codes(df)

# Проверяем результат
print("postal_code missing values:")
print(df['postal_code'].isnull().sum())

# Проверяем количество 'UNKNOWN' значений
print(" 'UNKNOWN' in postal_code:")
print((df['postal_code'] == 'UNKNOWN').sum())

postal_code missing values:
0
 'UNKNOWN' in postal_code:
0


/var/folders/9r/54xgf7zn6kddvpnb0lh4dgmc0000gn/T/ipykernel_94996/4069355567.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['postal_code'].fillna('UNKNOWN', inplace=True)


In [10]:
addresses.head()

,address_id,postal_code,district,region
0,1337,20083.0,MI,LOMBARDIA
1,1347,20090.0,MI,LOMBARDIA
2,1347,20090.0,MI,LOMBARDIA
3,1347,20090.0,MI,LOMBARDIA
4,1347,20090.0,MI,LOMBARDIA


### orders 

In [11]:
orders['purchase_datetime'] = pd.to_datetime(orders['purchase_datetime'])
orders

,order_id,customer_id,store_id,product_id,direction,gross_price,price_reduction,purchase_datetime
0,8fe4346b53d0c781b275a614175804de,23,49,35551922,1,125.40,0.0,2022-10-04 18:28:51
1,8fe4346b53d0c781b275a614175804de,23,49,30344384,1,8.30,0.0,2022-10-04 18:28:51
2,8fe4346b53d0c781b275a614175804de,23,49,33794292,1,1.99,0.0,2022-10-04 18:28:51
3,8fe4346b53d0c781b275a614175804de,23,49,33795776,1,5.10,0.0,2022-10-04 18:28:51
4,8fe4346b53d0c781b275a614175804de,23,49,36235381,1,76.05,0.0,2022-10-04 18:28:51
...,...,...,...,...,...,...,...,...
1039860,697c0078b69d9f8c119d8dd9a7952019,934581,29,35508942,1,33.60,0.0,2023-04-30 19:35:53
1039861,86c9c4fdb8829e2ab90a86a357763419,934620,44,36237691,1,11.99,0.0,2023-04-30 19:19:55
1039862,86c9c4fdb8829e2ab90a86a357763419,934620,44,81259479,1,9.99,0.0,2023-04-30 19:19:55
1039863,86f8ca9e4008249e5e0acc5ff23e2284,934660,46,34067775,1,1.90,0.0,2023-04-30 19:32:17


### tbl_products

In [12]:
products

,product_id,product_class
0,35996051,9
1,35980371,14
2,31276196,4
3,31276224,4
4,31456495,4
...,...,...
88533,36218763,3
88534,36219624,3
88535,36219631,3
88536,36219694,3


### tbl_customer_reviews

In [13]:
customer_rew

,review_id,customer_id,review_text
0,1,814256,I have bought several of the Vitality canned d...
1,2,457475,Product arrived labeled as Jumbo Salted Peanut...
2,3,104045,This is a confection that has been around a fe...
3,4,603662,If you are looking for the secret ingredient i...
4,5,739314,Great taffy at a great price. There was a wid...
...,...,...,...
105705,140942,224091,"We Love,Love our Keurig coffee maker .My husba..."
105706,140943,196666,This is supposed to be a light roast coffee bu...
105707,140944,618226,I have always been happy with my Kcups but thi...
105708,140945,844918,I ordered light roast coffee and was sent one ...


### labelled_reviews

In [14]:
reviews

,labelled_reviews_index,review_text,sentiment_label
0,0,I'm no bitters expert but I bought it as a gif...,neutral
1,1,"these are probably great in the right drinks, ...",neutral
2,2,I sent these to my dad for his bday and he sai...,positive
3,3,I purchased these as a gift for family member ...,positive
4,4,My wife bought me this sauce sampler for Chris...,neutral
...,...,...,...
462739,462739,Great for sesame chicken..this is a good if no...,positive
462740,462740,I'm disappointed with the flavor. The chocolat...,neutral
462741,462741,"These stars are small, so you can give 10-15 o...",positive
462742,462742,These are the BEST treats for training and rew...,positive


### tbl_customer_accounts

In [15]:
customer_acc

,customer_id,account_id,favorite_store,loyalty_type,loyatlty_status,activation_date
0,9557,746573,9,premium,1,2023-01-11
1,743090,776971,2,premium,1,2023-01-30
2,813156,791681,45,premium,1,2023-02-08
3,843392,825439,1,premium,1,2023-03-01
4,20476,815010,16,premium,1,2023-02-23
...,...,...,...,...,...,...
169489,828458,895971,47,standard,1,2023-04-09
169490,575891,525794,1,standard,1,2022-09-14
169491,316991,231471,14,standard,1,2022-04-15
169492,403638,329448,1,standard,1,2022-05-31
